## Evaluating Random Integral

### The given integral:

\begin{equation}
K_n = \displaystyle\int_a^b \frac{x^n}{z + ax} dx 
\end{equation} 

### The recursion relation:

\begin{equation}
K_{n+1} = \frac{1}{\alpha} \frac{b^{n+1} - a^{n+1}}{n+1} - \frac{z}{\alpha}K_n 
\end{equation} 

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
alpha = 2.0
z = 1.0

In [5]:
def func(n:int, x:float):
    return pow(x, n)/(z + alpha * x)

In [6]:
def upward_recursion(order:int, u_lim:int = 1, l_lim:int = 0):
    k = np.zeros(order+1)
    try:
        k_not = 1/alpha * np.log((z+alpha*u_lim)/(z+alpha*l_lim))
        k[0] = k_not
        if order <= 0 :
            return k_not
        
        for n in range(1, order + 1):
            k_next = 1/alpha * (pow(u_lim, n) - pow(l_lim, n))/n - (z/alpha) * k_not
            k[n] = k_next
            k_not = k_next

        return k

    except ZeroDivisionError:
        print("Enter non zero value for alpha and z")
        return 0.0


In [7]:
def standard_eval(order:int, steps:int = 10000, u_lim:int = 1, l_lim:int = 0):
    k_std = np.zeros(order+1)
    dx = (u_lim - l_lim)/steps
    for n in range (order+1):
        integral = 0.0
        for step in range(steps):
            x = (0.5 + step)*dx
            integral += func(n, x)
        k_std[n] = integral*dx
    return k_std

## Evaluate series:

\begin{equation}
Sum = K_{1} + K_2 + K_3 + ... + K_n
\end{equation}

In [8]:
def evaluate_series(terms:int):
    sum_up = sum(np.array(upward_recursion(terms)))
    sum_std = sum(np.array(standard_eval(terms)))
    return sum_up, sum_std

In [9]:
result_up, result_std = evaluate_series(10)
print(f'Error: {result_std - result_up}')

Error: -7.037037619284092e-09


Testing upward recursion for

\begin{equation} {\frac{\alpha}{z}} < 1 \end{equation}

In [10]:
alpha = 1e-4
result_up, result_std = evaluate_series(10)
print(f'Error: {result_std - result_up}')

Error: 1.1013182817544156e+27


In [11]:
def downward_recursion(order:int, u_lim:int = 1, l_lim:int = 0):
    epsilon = 1e-16
    sum_start = 0
    sum_end = int((np.log(epsilon))/(np.log(alpha/z)))
    k = np.zeros(order + 1)
    k_next = 0.0
    for m in range(sum_end + 1):
        k_next += (pow(-alpha, m)/pow(z, m+1)) * (pow(u_lim, order+m+2) - pow(l_lim, order+m+2))/(order+m+2)

    for n in range(order, -1, -1):
        k_prev =  1/z * (pow(u_lim, n+1) - pow(l_lim, n+1))/(n+1) - alpha/z * k_next
        k[n] = k_prev
        k_next = k_prev
    return k

In [12]:
alpha = 1e-4
sum_down = sum(np.array(downward_recursion(10)))
sum_std = sum(np.array(standard_eval(10)))
print(f'Downward Recursion error: {sum_std - sum_down}')

Downward Recursion error: -2.2497284923872485e-08


## Hybrid recursion

In [15]:
def hybrid_recursion(order:int, u_lim:int = 1, l_lim:int = 0):
    # z is assumed to be defined globally as 1.0
    alphas = np.linspace(1e-5, 4, 10)
    k_hyb = []
    k_std = []
    
    for current_alpha in alphas:
        # We must pass current_alpha or set it globally so the 
        # recursion functions see the changing value.
        global alpha
        alpha = current_alpha 
        
        # The stability criterion: use downward if alpha/z is small
        if abs(alpha/z) < 0.5:
            k_res = downward_recursion(order, u_lim, l_lim)
        else:
            k_res = upward_recursion(order, u_lim, l_lim)
            
        k_hyb.append(np.array(k_res))
        k_std.append(np.array(standard_eval(order, u_lim=u_lim, l_lim=l_lim)))

    return k_hyb, k_std

In [16]:
result, reference = hybrid_recursion(10)
for i in range(len(result)):
    print(f'Hybrid Error: {reference[i] - result[i]}')

Hybrid Error: [ 1.18793864e-14  8.38218384e-15 -8.33320357e-10 -1.24998326e-09
 -1.66664574e-09 -2.08330803e-09 -2.49997070e-09 -2.91663313e-09
 -3.33329565e-09 -3.74995827e-09 -4.16662069e-09]
Hybrid Error: [-9.64296420e-11  2.16964835e-10 -4.88162122e-10 -7.76620879e-10
 -1.06508100e-09 -1.35354032e-09 -1.64200049e-09 -1.93046008e-09
 -2.21891980e-09 -2.50737975e-09 -2.79583943e-09]
Hybrid Error: [-2.66566991e-10  2.99885727e-10 -3.37368383e-10 -5.57955376e-10
 -7.78543063e-10 -9.99130015e-10 -1.21971784e-09 -1.44030481e-09
 -1.66089251e-09 -1.88147935e-09 -2.10206719e-09]
Hybrid Error: [-4.53518778e-10  3.40136197e-10 -2.55100746e-10 -4.33672376e-10
 -6.12242881e-10 -7.90813970e-10 -9.69384656e-10 -1.14795561e-09
 -1.32652659e-09 -1.50509753e-09 -1.68366836e-09]
Hybrid Error: [-6.44742926e-10  3.62666758e-10 -2.03999401e-10 -3.53999233e-10
 -5.03998940e-10 -6.53998466e-10 -8.03998215e-10 -9.53997971e-10
 -1.10399760e-09 -1.25399736e-09 -1.40399699e-09]
Hybrid Error: [-8.36748115e-10